In [1]:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_openai import OpenAIEmbeddings
from pathlib import Path
from chromadb.utils import embedding_functions
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [15]:
load_dotenv()
knowledge_base_paths = Path("KnowledgeBase")
files_path = [f for f in knowledge_base_paths.iterdir() if f.is_file()]

docs = [PyPDFLoader(path).load() for path in files_path]
docs_list = [item for sublist in docs for item in sublist]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
chunks = splitter.split_documents(docs_list)



In [16]:
len(chunks)

428

In [17]:
chunks[20]

Document(metadata={'source': 'KnowledgeBase/asrin-getirdigi-tereddutler-1-M-F-Gulen.pdf', 'page': 10}, page_content='“Kur’ân, olmuş ve olacak her şeyden bahsediyor.” \ndiyorlar. Bu doğru mudur? Doğru ise, günümüzdeki \nbir kısım fen ve tekniğe ait meseleleri de bunun içinde \nmütalâa edebilir miyiz?\nYüce Yaratıcı’nın, insanoğlunun öğrenmesine müsaade ettiği ve onun maddî-\nmânevî terakkisine vesile kıldığı her şeyden icmâlen bahsetmesi doğrudur.\nAncak, Allah’ın (celle celâluhu) müsaade etmediği ve insanoğlunun da dünya\nve ahiret hayatına bir faydası dokunmayan şeylerden söz etmesi, hele tafsilde\nbulunması asla bahis mevzuu değildir. Zira, böyle bir şeyi kabullenmek, hikmet\ndolu bir kitaba abes isnat etmek olur ki, o mukaddes beyan bu türlü faydasızlık\nve abesiyetten çok muallâdır!...\nKur’ân’ın ele alıp tahlile tâbi tuttuğu şeylerde, takip ettiği bir yol vardır ki, o\nyol bilinmediği zaman, tahlilci çok defa hayal kırıklığına uğrayabilir; yani,\naradığını onda bulamayabilir.\nBir

In [14]:
embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
res = embedding_function.embed_query("Hallo")

In [7]:
len(res)

3072

In [18]:
vectorstore = Chroma.from_documents(
documents=chunks, 
embedding=embedding_function,
collection_name="asrin-getirdigi-tereddutler", persist_directory="./.chromadb",
collection_metadata={"embeeding_model":"text-embedding-3-large",
"chunk_size":1024,
"chunk_overlap":100})



In [19]:
vectorstore._collection.count()

428

In [20]:
len(vectorstore._collection.peek(1)['embeddings'][0])

3072

In [22]:
vectorstore = Chroma( 
embedding_function=embedding_function,
collection_name="asrin-getirdigi-tereddutler", 
persist_directory="./.chromadb",
collection_metadata={"embeeding_model":"text-embedding-3-large",
"chunk_size":1024,
"chunk_overlap":100})


In [44]:
Allah her şeyi yarattı, -hâşâ- O’nu kim yarattı?query = ""
query2 ="Nürnbergde hava nasil?"

In [45]:
results = vectorstore.similarity_search_with_score(
    query2, k=3
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")


* [SIM=1.530767] de istifham ortaya çıkacaktır. Meselâ, yağmur, buluta bağlı; bulut, zâit-nâkıs
(artı-eksi) habbeciklere, onlar buharlaşma hâdisesine, o da suların
mevcudiyetine ve nihayet o da suyu meydana getiren unsurlara... Böylece
sebepler zinciri, belki birkaç adım daha ilerleyerek devam eder durur; ama
durduğu yerde yine “Şöyle de olabilir, böyle de” diyerek insan kendini
faraziyelerin kucağında hisseder ve onlarla tatmin olmaya çalışır. Bu ise,
fevkalâde bir nizam; bir âhenk ve birbiriyle münasebet içinde, bir hikmet eliyle
meydana geldiği sezilen bütün eşyayı çocuk hezeyanlarıyla izah etmeye
yeltenmekten başka, bir de ilimlerin ufkunu ve hedefini karartmak demektir.
Oysaki, her netice için mutlaka mâkul bir sebebe ihtiyaç vardır. Gayr-i mâkul
ve gayr-i mantıkî sebeplerin uzayıp gitmesi, uzayıp gitmenin kerameti olarak [{'page': 28, 'source': 'KnowledgeBase/asrin-getirdigi-tereddutler-1-M-F-Gulen.pdf'}]
* [SIM=1.538648] bunun altında, üstünde hava bulunmadığı sözünü
1
, geleceğ

In [84]:
import chromadb

client = chromadb.PersistentClient(path=".chromadb")


In [88]:
messages_collection = client.get_or_create_collection("chat_messages")    

In [87]:
client.list_collections()

[Collection(name=asrin-getirdigi-tereddutler), Collection(name=chat_messages)]

In [98]:
col = [{"input": "chat_messages", "output": "chat_messages"}]

In [97]:
messages_collection.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['test'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}